<a href="https://colab.research.google.com/github/sanjanagarai/InterIIT-MLtask1/blob/main/Alternative_approaches_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 ALTERNATE APPROACHES TO THE PROBLEM

Installing libraries

In [ ]:
!pip install transformers
!pip install sentence_transformers

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from transformers import FlaubertTokenizer, FlaubertForSequenceClassification, AdamW

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

Loading the dataset

In [ ]:
df = pd.read_csv('cgptsamples-1.csv')

Defining the model

In [ ]:
model_name = "bert-base-nli-mean-tokens"
model=SentenceTransformer(model_name)

## DATA AUGMENTATION

negative samplING using cosine similarity

In [ ]:
#defining the function
def generate_negative_samples(df, model, sentence_vecs1, sentence_vecs2):
    negative_samples = []

    for i in range(len(df)):
      negative_samples.append((df['premise'][i], df['hypothesis'][i], 1))

    for premise_idx, premise in enumerate(df['premise']):
        premise_vec = sentence_vecs1[premise_idx].reshape(1, -1)
        hypothesis_vecs = sentence_vecs2

        # Compute cosine similarities between the premise and all hypotheses
        similarities = cosine_similarity(premise_vec, hypothesis_vecs)[0]

        # Find the indices of 2 hypotheses with the least similarity
        least_similar_indices = np.argpartition(similarities, 2)[:2]

        # Append these hypotheses to negative_samples with label 0
        for idx in least_similar_indices:
            negative_samples.append((premise, df['hypothesis'][idx], 0))

    return negative_samples

In [ ]:
sentences1 = df['premise']
sentences2 = df['hypothesis']

sentences1 = model.encode(sentences1)
sentences2 = model.encode(sentences2)

# Generate negative samples
negative_samples = generate_negative_samples(df, model, sentences1, sentences2)

# Create a DataFrame from the negative_samples list
df_negative_samples = pd.DataFrame(negative_samples, columns=['premise', 'hypothesis', 'label'])

# Specify the output file path
output_file_path = 'augmented_cgptsamples.csv'

# Save the DataFrame to a CSV file
df_negative_samples.to_csv(output_file_path, index=False)

print(f"Negative samples saved to {output_file_path}")

Negative samples saved to augmented_cgptsamples.csv


In [ ]:
df = pd.read_csv('augmented_cgptsamples.csv')

## ALT APPROACH 1

Creating similar sentences of the premise and hypothesis using the GloVe vocabulary and Genism module to make the dataset more robust

---



---



DRAWBACK: Similar sentences created don't have the right grammar and structure and hence, caused degraded training of the model

In [ ]:
import gensim.downloader as api
import random

# Load GloVe vectors outside the function
glove_model = api.load("glove-wiki-gigaword-50")

def generate_similar_sentences(input_sentence, num_similar=2):
    # Tokenize the input sentence
    tokens = input_sentence.split()

    # Generate similar sentences
    similar_sentences = []

    for token in tokens:
        if token in glove_model:
            # Find similar words using GloVe vectors
            similar_words = glove_model.most_similar(token, topn=5)
            for similar_word, _ in similar_words:
                # Replace the original word with a similar word
                new_tokens = tokens[:]
                new_tokens[new_tokens.index(token)] = similar_word
                similar_sentences.append(" ".join(new_tokens))

    # Shuffle the list of similar sentences
    random.shuffle(similar_sentences)

    # Return the specified number of similar sentences
    return similar_sentences[:num_similar]

# Create a list to store expanded data
expanded_data = []

# Iterate through each row in the dataset
for _, row in df.iterrows():
    premise = row['premise']
    hypothesis = row['hypothesis']
    label = row['label']

    # Generate 5 similar premises and hypotheses
    similar_premises = generate_similar_sentences(premise)
    similar_hypotheses = generate_similar_sentences(hypothesis)

    # Create new rows for each similar pair
    for similar_premise, similar_hypothesis in zip(similar_premises, similar_hypotheses):
        expanded_data.append({'premise': similar_premise, 'hypothesis': similar_hypothesis, 'label': label})

# Create a new DataFrame from the expanded data
expanded_df = pd.DataFrame(expanded_data)

# Save the expanded dataset to a new CSV file
expanded_df.to_csv("expanded_cgptsamples.csv", index=False)

[==================================================] 100.0% 66.0/66.0MB downloaded


A new dataset - 'expanded_cgptsamples.csv' with similar sentence entries has been created

In [ ]:
# Load your dataset from 'augmented_cgptsamples.csv' (replace with your actual file path)
df = pd.read_csv('expanded_cgptsamples.csv')

# Split the dataset into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Identify common rows between train and test datasets
common_rows = pd.merge(train_df, test_df, on=['premise', 'hypothesis'], how='inner')

# Remove common rows from the test dataset
test_df = test_df[~test_df.index.isin(common_rows.index)]

# Save the train and test sets to CSV files
train_df.to_csv('train_dataset.csv', index=False)  # Save train dataset to 'train_dataset.csv'
test_df.to_csv('test_dataset.csv', index=False)    # Save updated test dataset to 'test_dataset.csv'


## ALT APPROACH 2

Training the model using GPT 2 transformer

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW

# Load your dataset (modify the file path)
df = pd.read_csv("train_dataset.csv")

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        premise = str(self.data.loc[index, 'premise'])
        hypothesis = str(self.data.loc[index, 'hypothesis'])
        label = int(self.data.loc[index, 'label'])

        # Combine premise and hypothesis with a separator
        text = premise + " [SEP] " + hypothesis

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the tokenizer and the model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2).to(device)

# Set hyperparameters
batch_size = 8
max_length = 128
learning_rate = 2e-5
num_epochs = 3

# Create the dataset and dataloader
dataset = CustomDataset(df, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Split the data into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Set the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {average_loss:.4f}")

# Evaluation on the validation set
model.eval()
all_predictions = []
true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predicted_labels = torch.argmax(logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, all_predictions)
report = classification_report(true_labels, all_predictions, target_names=["Not Satisfying", "Satisfying"])

print("Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(report)

# Save the trained model
model.save_pretrained("trained_gpt_model")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

## ALT APPROACH 3

Using ChatGPT to create similar sentences by using the ChatGPT API

In [ ]:
import openai
import random

# Set your OpenAI API key
api_key = "YOUR_API_KEY"

# Initialize the OpenAI API client
openai.api_key = api_key

def generate_similar_sentences(input_sentence, num_similar=2):
    # Generate similar sentences using ChatGPT
    similar_sentences = []

    for _ in range(num_similar):
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=f"Make a sentence similar to: '{input_sentence}'",
            max_tokens=50,  # Adjust the max_tokens as needed
            stop=None,  # You can specify a list of strings to stop generation if necessary
            temperature=0.7  # Adjust temperature for randomness
        )

        generated_sentence = response.choices[0].text.strip()
        similar_sentences.append(generated_sentence)

    return similar_sentences

# Iterate through each row in the dataset
for _, row in df.iterrows():
    premise = row['premise']
    hypothesis = row['hypothesis']
    label = row['label']

    # Generate 5 similar premises and hypotheses
    similar_premises = generate_similar_sentences(premise)
    similar_hypotheses = generate_similar_sentences(hypothesis)

    # Create new rows for each similar pair
    for similar_premise, similar_hypothesis in zip(similar_premises, similar_hypotheses):
        expanded_data.append({'premise': similar_premise, 'hypothesis': similar_hypothesis, 'label': label})

# Create a new DataFrame from the expanded data
expanded_df = pd.DataFrame(expanded_data)

# Save the expanded dataset to a new CSV file
expanded_df.to_csv("expanded_cgptsamples.csv", index=False)


